In [1]:
!pip install -U torchaudio librosa jiwer datasets transformers huggingface_hub evaluate speechbrain moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import pipeline
import evaluate
from datasets import load_dataset, Audio
import re
from pandas import DataFrame

import torch
import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement, SepformerSeparation, VAD

import moviepy.editor as mp
import numpy as np

SAMPLE_RATE = 16_000;

/usr/local/lib/python3.9/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


The notebook reads a single audio file, detects pauses in the speech, splits the file by these pauses and runs ASR on each of the parts.

The audio can be either 1-channel or 2 channels, and for the later case, the code must be adjusted.

In [3]:
# This cell is needed to work with Drive from Colab

from google.colab import drive
drive.mount("/content/gdrive")
DRIVE_PATH = "/content/gdrive/MyDrive/NNProjects/FEM/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
VIDEO_PATH = f"{DRIVE_PATH}/testing_FEM.mp4"
audio = mp.VideoFileClip(VIDEO_PATH).audio
orig_freq = audio.fps

In [6]:
audio_arr = audio.to_soundarray()
# audio_arr = audio_arr[:,:1] # In our case, the channels have approximately the same sound, so we pick only one
audio_arr = torchaudio.functional.resample(torch.Tensor(audio_arr.T), orig_freq=orig_freq, new_freq=SAMPLE_RATE)

  return stacker(self.iter_chunks(fps=fps, quantize=quantize,



In [7]:
# Voice activity detection only works with a stored file, so I save a down-sampled version
# It also requires a single-channel audio, so here I subset only the first channel. In this example
# the channels have the same info, so the second one can be ignored. In practice we may need to run it twice.
audio_path = f"{DRIVE_PATH}/testing_FEM_downsampled.wav"
torchaudio.save(audio_path, src=audio_arr[:1,:], sample_rate=SAMPLE_RATE)

## Voice activity detection

In [9]:
vad = VAD.from_hparams(source="speechbrain/vad-crdnn-libriparty")
boundaries = vad.get_speech_segments(audio_path, apply_energy_VAD=True) # apply_energy_VAD gives finer segmentation
boundaries = boundaries[~(np.arange(boundaries.shape[0]) % 2).astype(bool),:] # Only even ids correspond to speech

boundary_ids = (boundaries * SAMPLE_RATE).detach().numpy().astype(int)
region_df = DataFrame(boundaries, columns=["start", "end"])
region_df

Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:862.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]



,start,end
0,4.450000,5.760000
1,12.960000,17.209999
2,20.039999,23.119999
3,30.799999,32.070000


## Transcription

In [10]:
model = pipeline(model="Akashpb13/Hausa_xlsr", device=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
region_df["text"] = [model(audio_arr[0, s:e].numpy())['text'] for s,e in boundary_ids]
region_df["channel"] = 0 # In the future, perhaps we'll have multiple channels here
region_df

,start,end,text,channel
0,4.450000,5.760000,sarin yali,0
1,12.960000,17.209999,musamma tsakanin mutane masu bin tuffarkin add...,0
2,20.039999,23.119999,za ka gama gidanciya taramata fiyi ra ɗaya,0
3,30.799999,32.070000,ufafi mai kyau,0


In [12]:
region_df.to_csv(f"{DRIVE_PATH}/transcription.csv")